### To view the tensorboard: 
    1: tensorboard --logdir ray_results 
    2: see http://localhost:6006/ in browser

In [1]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
from ray import tune

## 0: RLlib Training APIs: 
1: At a high level, RLlib provides an Trainer class which holds a policy for environment interaction. Through the trainer interface, the policy can be trained, checkpointed, or an action computed. In multi-agent training, the trainer manages the querying and optimization of multiple policies at once.

2: rllib train --run DQN --env CartPole-v0  --config '{"num_workers": 8}'
    To see the tensorboard: tensorboard --logdir=~/ray_results

3: rllib rollout ~/ray_results/default/DQN_CartPole-v0_0upjmdgr0/checkpoint_1/checkpoint-1 \
    --run DQN --env CartPole-v0 --steps 10000

4: Loading and restoring a trained agent from a checkpoint is simple:
    
    agent = ppo.PPOTrainer(config=config, env=env_class)
    agent.restore(checkpoint_path)
    
5: Computing Actions

The simplest way to programmatically compute actions from a trained agent is to use trainer.compute_action(). This method preprocesses and filters the observation before passing it to the agent policy. Here is a simple example of testing a trained agent for one episode:

    # instantiate env class
    env = env_class(env_config)

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward

In [2]:
ray.shutdown()
ray.init()

2021-10-15 10:50:05,689	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.0.42',
 'raylet_ip_address': '192.168.0.42',
 'redis_address': '192.168.0.42:27869',
 'object_store_address': '/tmp/ray/session_2021-10-15_10-50-04_282399_1061/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-10-15_10-50-04_282399_1061/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-10-15_10-50-04_282399_1061',
 'metrics_export_port': 57969,
 'node_id': '7d4c98cc3697a0840fae3f832d847c4bc04e4aae4ee62657176f6e19'}

#### 1 Example of Traing a PPO Agent

In [4]:
config = ppo.DEFAULT_CONFIG.copy()
config['num_gpus'] = 0
config['num_workers'] = 2
trainer = ppo.PPOTrainer(config = config, env='CartPole-v0') 

for i in range(30):
    result = trainer.train()
    if i % 10 ==0:
        checkpoint = trainer.save()
        print('checkpoint saved')
        

2021-09-30 21:13:18,338	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-09-30 21:13:18,341	INFO ppo.py:158 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-09-30 21:13:18,342	INFO trainer.py:726 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-09-30 21:13:23,657	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2021-09-30 21:13:24,896	WARNING deprecation.py:38 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


checkpoint saved
checkpoint saved
checkpoint saved


#### 2 Example of Using Tune

In [3]:
alg = 'PPO'
tune.run(alg,
    stop={'episode_reward_mean':200},
    config={
        'env':'CartPole-v0',
        'num_gpus':0,
        'num_workers':2,
        'lr':tune.grid_search([.01,.001,.0001])     
    }
)

Trial name,status,loc,lr
PPO_CartPole-v0_97f47_00000,PENDING,,0.01
PPO_CartPole-v0_97f47_00001,PENDING,,0.001
PPO_CartPole-v0_97f47_00002,PENDING,,0.0001


(pid=1096) 2021-10-15 10:50:22,046	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=1096) 2021-10-15 10:50:22,046	INFO ppo.py:158 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=1096) 2021-10-15 10:50:22,046	INFO trainer.py:726 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=1100) 2021-10-15 10:50:22,046	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=1100) 2021-10-15 10:50:22,046	INFO ppo.py:158 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=1100) 2021-10-15 10:50:22,046	INFO trainer.py:726 -- Current log_level is WARN. For more information, set 'log_level': 'INFO'

Result for PPO_CartPole-v0_97f47_00001:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2021-10-15_10-50-31
  done: false
  episode_len_mean: 23.023121387283236
  episode_media: {}
  episode_reward_max: 64.0
  episode_reward_mean: 23.023121387283236
  episode_reward_min: 8.0
  episodes_this_iter: 173
  episodes_total: 173
  experiment_id: a7d670197b8449aca715ca10f26be361
  hostname: Liweis-iMac.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0010000000474974513
          entropy: 0.6616495251655579
          entropy_coeff: 0.0
          kl: 0.032735127955675125
          model: {}
          policy_loss: -0.05019322782754898
          total_loss: 91.09257507324219
          vf_explained_var: 0.28667253255844116
          vf_loss: 91.13622283935547
    num_agent_steps_sampled: 4000
    num_agent_steps_trained: 4000
    num_steps_sampled: 4000
    num_steps_trained

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_97f47_00000,RUNNING,,0.01,,,,,,,
PPO_CartPole-v0_97f47_00001,RUNNING,192.168.0.42:1099,0.001,1,3.87207,4000,23.0231,64,8,23.0231
PPO_CartPole-v0_97f47_00002,RUNNING,,0.0001,,,,,,,


Result for PPO_CartPole-v0_97f47_00000:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2021-10-15_10-50-31
  done: false
  episode_len_mean: 23.53846153846154
  episode_media: {}
  episode_reward_max: 80.0
  episode_reward_mean: 23.53846153846154
  episode_reward_min: 9.0
  episodes_this_iter: 169
  episodes_total: 169
  experiment_id: f750806d183f44688b827b772469e4d4
  hostname: Liweis-iMac.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.009999999776482582
          entropy: 0.6601420640945435
          entropy_coeff: 0.0
          kl: 0.03442293405532837
          model: {}
          policy_loss: -0.03388642519712448
          total_loss: 99.54802703857422
          vf_explained_var: 0.3093220889568329
          vf_loss: 99.57502746582031
    num_agent_steps_sampled: 4000
    num_agent_steps_trained: 4000
    num_steps_sampled: 4000
    num_steps_trained: 400

...

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_97f47_00000,RUNNING,192.168.0.42:1096,0.01,289,2452.95,1156000,195.87,200,139,195.87
PPO_CartPole-v0_97f47_00001,TERMINATED,,0.001,15,53.5871,60000,200,200,200,200
PPO_CartPole-v0_97f47_00002,TERMINATED,,0.0001,17,59.4017,68000,200,200,200,200


Result for PPO_CartPole-v0_97f47_00000:
  agent_timesteps_total: 1168000
  custom_metrics: {}
  date: 2021-10-15_11-31-31
  done: true
  episode_len_mean: 200.0
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 200.0
  episode_reward_min: 200.0
  episodes_this_iter: 20
  episodes_total: 6251
  experiment_id: f750806d183f44688b827b772469e4d4
  hostname: Liweis-iMac.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.016103263944387436
          cur_lr: 0.009999999776482582
          entropy: 0.008555091917514801
          entropy_coeff: 0.0
          kl: 0.10656091570854187
          model: {}
          policy_loss: 0.005454843398183584
          total_loss: 164.05479431152344
          vf_explained_var: 0.5440415740013123
          vf_loss: 164.047607421875
    num_agent_steps_sampled: 1168000
    num_agent_steps_trained: 1168000
    num_steps_sampled: 1168000
    num_steps_trained: 1168000
  

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_97f47_00000,TERMINATED,,0.01,292,2459.61,1168000,200,200,200,200
PPO_CartPole-v0_97f47_00001,TERMINATED,,0.001,15,53.5871,60000,200,200,200,200
PPO_CartPole-v0_97f47_00002,TERMINATED,,0.0001,17,59.4017,68000,200,200,200,200


Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_97f47_00000,TERMINATED,,0.01,292,2459.61,1168000,200,200,200,200
PPO_CartPole-v0_97f47_00001,TERMINATED,,0.001,15,53.5871,60000,200,200,200,200
PPO_CartPole-v0_97f47_00002,TERMINATED,,0.0001,17,59.4017,68000,200,200,200,200


2021-10-15 11:31:32,087	INFO tune.py:561 -- Total run time: 2474.84 seconds (2474.02 seconds for the tuning loop).


In [4]:
alg = 'DDPG'
tune.run(alg,
    stop={"training_iteration": 30},
    config={
        'env':'Pendulum-v0',
        'num_gpus':0,
        'num_workers':2,
        'lr':tune.grid_search([.001,])     
    }
)


Trial name,status,loc,lr
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,,0.001


(pid=17553) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=17553) Instructions for updating:
(pid=17553) non-resource variables are not supported in the long term
(pid=17553) 2021-07-13 20:54:31,552	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=17553) 2021-07-13 20:54:31,552	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=17556) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=17556) Instructions for updating:
(pid=17556

Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_20-54-41
  done: false
  episode_len_mean: 200.0
  episode_reward_max: -781.1207639597676
  episode_reward_mean: -1131.8902231793475
  episode_reward_min: -1410.9375467664963
  episodes_this_iter: 6
  episodes_total: 6
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 1500
    learner:
      default_policy:
        max_q: 0.1796478033065796
        mean_q: -0.10519210994243622
        min_q: -0.6069953441619873
        model: {}
    num_steps_sampled: 1500
    num_steps_trained: 256
    num_target_updates: 1
  iterations_since_restore: 1
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 25.799999999999997
    ram_util_percent: 63.55
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.0973585918644

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,192.168.0.23:17553,0.001,1,2.53136,1500,-1131.89,-781.121,-1410.94,200


Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_20-54-58
  done: false
  episode_len_mean: 200.0
  episode_reward_max: -781.1207639597676
  episode_reward_mean: -1331.3202113249129
  episode_reward_min: -1796.9534032278605
  episodes_this_iter: 6
  episodes_total: 12
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 2500
    learner:
      default_policy:
        max_q: -0.50157630443573
        mean_q: -11.473435401916504
        min_q: -24.651611328125
        model: {}
    num_steps_sampled: 2500
    num_steps_trained: 128256
    num_target_updates: 501
  iterations_since_restore: 2
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 23.545833333333334
    ram_util_percent: 63.48333333333334
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.

...

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,RUNNING,192.168.0.23:17553,0.001,29,507.24,29500,-461.236,-1.11165,-1519.85,200


Result for DDPG_Pendulum-v0_6b1dd_00000:
  custom_metrics: {}
  date: 2021-07-13_21-03-25
  done: true
  episode_len_mean: 200.0
  episode_reward_max: -1.1116457547643066
  episode_reward_mean: -444.3119269581459
  episode_reward_min: -1524.5879513507703
  episodes_this_iter: 6
  episodes_total: 152
  experiment_id: 43107a19371e49529eb221515032f0aa
  hostname: Mingjuns-MacBook-Pro.local
  info:
    last_target_update_ts: 30500
    learner:
      default_policy:
        max_q: 5.001977920532227
        mean_q: -80.31802368164062
        min_q: -147.42247009277344
        model: {}
    num_steps_sampled: 30500
    num_steps_trained: 3712256
    num_target_updates: 14501
  iterations_since_restore: 30
  node_ip: 192.168.0.23
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 25.751851851851853
    ram_util_percent: 63.059259259259264
  pid: 17553
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processi

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,TERMINATED,,0.001,30,526.471,30500,-444.312,-1.11165,-1524.59,200


Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DDPG_Pendulum-v0_6b1dd_00000,TERMINATED,,0.001,30,526.471,30500,-444.312,-1.11165,-1524.59,200


2021-07-13 21:03:26,318	INFO tune.py:448 -- Total run time: 540.24 seconds (539.55 seconds for the tuning loop).


## 1: RLlib Environments

1: RLlib works with several different types of environments, including OpenAI Gym, user-defined, multi-agent, and also batched environments.

2: RLlib uses Gym as its environment interface for single-agent training.



#### 1: Configuring Environments

    https://github.com/ray-project/ray/blob/master/rllib/examples/custom_env.py

In [3]:
import gym, ray
from ray.rllib.agents import ppo

class MyEnv(gym.Env):
    def __init__(self, env_config):
        self.action_space = <gym.Space>
        self.observation_space = <gym.Space>
    def reset(self):
        return <obs>
    def step(self, action):
        return <obs>, <reward: float>, <done: bool>, <info: dict>

ray.init()
trainer = ppo.PPOTrainer(env=MyEnv, config={
    "env_config": {},  # config to pass to env class
})

while True:
    print(trainer.train())

SyntaxError: invalid syntax (<ipython-input-3-7352217a18a1>, line 6)